## Agentic AI (ReAct) with PhiData

### LLM Used - Granite3.1-8B

In this notebook, you will learn how to create and deploy a ReAct (Reasoning + Acting) agent capable of solving complex tasks by combining logical reasoning with external tool usage. The agent will use a structured loop of Thought → Action → Observation to iteratively reason through problems, gather data, and provide solutions.

> NOTE: This notebook requires vLLM 0.6.4 onwards that includes [Tool Calling / Function Calling](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#ibm-granite).
    A custom Service Runtime is used with the following parameters:
```
    - --enable-auto-tool-choice
    - --tool-call-parser
    - "granite"
    - --chat-template
    - "/app/data/template/tool_chat_template_granite.jinja"
```

### 1. Agentic AI Overview 

An Agent is an autonomous system that leverages Large Language Models (LLMs) to perform tasks by understanding, reasoning, planning, and executing actions with minimal human intervention. AI agents are designed to break down complex problems into manageable steps, utilizing tools, accessing memory, and adapting their behavior based on the provided context.

At its core, an agent is structured to:

* Receive a Task: The agent takes input from the user, such as a question or command.
* Plan a Solution: The agent decomposes the problem, chooses appropriate tools, and reasons through possible solutions.
* Execute the Plan: It performs actions, such as retrieving information, using tools, or generating responses based on the devised plan.
* Deliver Results: Finally, it presents the solution or output in a structured, actionable format.

![LLM Tools](../../content/modules/ROOT/assets/images/04/04-02-react-diagram.png)

### 2. Agentic AI Framework Used - Phidata

**Phidata** is a lightweight Python framework for building, deploying, and managing data workflows and cloud infrastructure. It simplifies data pipelines and cloud deployments with a declarative, code-first approach, enabling seamless integration with popular tools and providers. Phidata’s modular design supports dynamic workflows and real-time processing, making it ideal for modern data engineering.

### 3. Setup and Import Libraries

To get started, you'll need to install and import a few Python libraries. Run the following command to install them:

In [1]:
!pip install -q duckduckgo_search==7.1.0 yfinance
!pip install -qU phidata openai


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports
import os
import json
import getpass

from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.yfinance import YFinanceTools
from phi.tools.duckduckgo import DuckDuckGo

from typing_extensions import TypedDict
from typing import Annotated


### 4. Model Configuration

We will start by creating an llm instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model.


#### 4.1 Define the Inference Model Server specifics

In [3]:
INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite-3-8b-instruct"
API_KEY= os.getenv('API_KEY_GRANITE')

#### 4.2 Create the LLM instance

In [4]:
model = OpenAIChat(
    id=MODEL_NAME,
    base_url=f"{INFERENCE_SERVER_URL}/v1",
    api_key=API_KEY
)

### 5. Define tools

First, we will configure the tool by initializing the YFinance functionality. This tool will serve as the primary resource for retrieving up-to-date financial data, such as stock prices, analyst recommendations, company details, and news.

Additionally, for scenarios where the agent needs to process complex or real-time financial data beyond basic queries, YFinance provides specialized methods to retrieve and analyze relevant information effectively. By using this tool, we can empower the agent to provide precise and actionable insights directly from the financial markets.

Here's how to configure the YFinance tool:

In [5]:
yfinance = YFinanceTools(stock_price=True, 
                         analyst_recommendations=True,
                         company_info=True,
                         company_news=True)

On the other hand we will use as well DuckDuckGo tool to provide to our model searching capabilities:

In [6]:
duckduckgo = DuckDuckGo()

### 6. Creating a Phidata Agent

In this section, we will build a Phidata agent that autonomously reasons through tasks and selects the appropriate tools to solve problems. The agent will follow a structured process to deliver actionable results.

In [7]:
agent = Agent(
    model=model,
    tools=[yfinance],
    show_tool_calls=True,
    markdown=False,
    debug_mode=False,
)

#### 6.1 Query the agent with tools included

Here, we query our Phidata agent to retrieve the current stock prices for JP Morgan (JPM) and Tesla (TSLA). To accomplish this, we’ve equipped the agent with the YFinance Tool, enabling it to access real-time financial data.

Let’s see the agent in action!

In [8]:
agent.print_response("What is the stock price of JPM and TSLA", stream=True)

▰▰▰▰▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ What is the stock price of JPM and TSLA                                                                         ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (4.4s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ <tool_call>                                                                                                     ┃
┃ Running:                                                                                                        ┃
┃  - get_current_stock_price(symbol=JPM)                                                                          ┃
┃  - get_current_stock_price(symbol=TSLA)                                                                         ┃
┃                                                                                                                 ┃
┃ The current stock price of JPM is $269.35 and the current stock price of TSLA is $389.62.                       ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

#### 6.2 Summarization of Analyst Recommendations

Next, we want to retrieve and summarize analyst recommendations for IBM. The agent will process the data and provide a concise summary, streaming the response in real-time for better interactivity.

In [9]:
agent.print_response("Summarize analyst recommendations for IBM", stream=True)

▰▰▰▰▰▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Summarize analyst recommendations for IBM                                                                       ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (12.3s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ <tool_call>                                                                                                     ┃
┃ Running:                                                                                                        ┃
┃  - get_analyst_recommendations(symbol=IBM)                                                                      ┃
┃                                                                                                                 ┃
┃ Here are the recent analyst recommendations for IBM:                                                            ┃
┃                                                                                                                 ┃
┃ - For the last 0 months, out of 15 analysts, 2 recommend a strong buy, 5 recommend a buy, 8 suggest holding, 3  ┃
┃ recommend selling, and 1 advises selling with conviction.                                                       ┃
┃ - For the last -1 month, out of 15 analysts, 2 recommend a strong buy, 5 recommend a buy, 8 suggest holding, 3  ┃
┃ recommend selling, and 1 advises selling with conviction.                                                       ┃
┃ - For the last -2 months, out of 15 analysts, 2 recommend a strong buy, 5 recommend a buy, 9 suggest holding, 3 ┃
┃ recommend selling, and 1 advises selling with conviction.                                                       ┃
┃ - For the last -3 months, out of 15 analysts, 2 recommend a strong buy, 5 recommend a buy, 9 suggest holding, 3 ┃
┃ recommend selling, and 1 advises selling with conviction.                                                       ┃
┃                                                                                                                 ┃
┃ The overall sentiment among the analysts remains positive, with slightly more recommending a buy or hold. There ┃
┃ hasn't been a significant change in the recommendations over the past three months.                             ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

### 7. Building our own Finance Report Agent

Here, we configure a finance report agent to compare JP Morgan (JPM) and Tesla (TSLA) using the YFinance tool for real-time financial data. The agent is set to use tables for clarity and provides a detailed reasoning process to explain its analysis. With streaming enabled, we’ll generate a real-time, structured comparative report. Let’s see the output!

In [10]:
finance_report_agent = Agent(
    model=model,
    tools=[yfinance],
    instructions=["Use tables where possible"],
    show_tool_calls=True,
    markdown=True,
    debug_mode=False,
)

finance_report_agent.print_response("Write a report comparing JPM to TSLA", stream=True, show_full_reasoning=True)

▰▰▱▱▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Write a report comparing JPM to TSLA                                                                            ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (19.6s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃   Metric                    JPM (JPMorgan Chase)                      TSLA (Tesla)                              ┃
┃  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  ┃
┃   Current Stock Price       (Call the function                        (Call the function                        ┃
┃                             get_current_stock_price with the symbol   get_current_stock_price with the symbol   ┃
┃                             "JPM")                                    "TSLA")                                   ┃
┃   Company Info              (Call the function get_company_info       (Call the function get_company_info       ┃
┃                             with the symbol "JPM")                    with the symbol "TSLA")                   ┃
┃   Analyst Recommendations   (Call the function                        (Call the function                        ┃
┃                             get_analyst_recommendations with the      get_analyst_recommendations with the      ┃
┃                             symbol "JPM")                             symbol "TSLA")                            ┃
┃   Recent News               (Call the function get_company_news       (Call the function get_company_news       ┃
┃                             with the symbol "JPM" and                 with the symbol "TSLA" and                ┃
┃                             num_stories=3)                            num_stories=3)                            ┃
┃                                                                                                                 ┃
┃                                                                                                                 ┃
┃ In the report, you would fill in the values obtained from the APIs. JPM (JPMorgan Chase) and TSLA (Tesla) are   ┃
┃ both prominent companies, yet they operate in different sectors - banking and automotive, respectively.         ┃
┃                                                                                                                 ┃
┃ JPMorgan Chase, boasting a strong balance sheet and diversified business model, is known for its financial      ┃
┃ services and investment banking. Tesla, an industry leader in electric vehicles, has been transforming the      ┃
┃ automotive sector with its innovative products and sustainable energy solutions.                                ┃
┃                                                                                                                 ┃
┃ Comparing their current stock prices will give us an insight into investor sentiment towards each company's     ┃
┃ stock at the time of the report. Analyzing their company information will allow us to understand the core       ┃
┃ business operations and market stands of both companies.                                                        ┃
┃                                   

### 8. Using Multiple Tools with our Agent

In this step, we configure a multi-tool agent that combines the DuckDuckGo tool for web search and the YFinance tool for retrieving financial data. This setup allows the agent to handle both general search queries and financial-specific tasks in a single prompt.

The agent is instructed to use tables for presenting data for better clarity. In this example, we’ll ask the agent to fetch the stock price for JP Morgan (JPM) and simultaneously search for current trends related to it.

Let’s see how the agent performs with multiple tools working together!

In [11]:
multi_tool_agent = Agent(
    model=model,
    tools=[DuckDuckGo(), yfinance],
    instructions=["Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

In [12]:
agent.print_response("Get the stock price from JPM and search what the trends are", stream=True)

▰▰▰▰▰▰▰ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Get the stock price from JPM and search what the trends are                                                     ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (13.4s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ <tool_call>                                                                                                     ┃
┃ Running:                                                                                                        ┃
┃  - get_current_stock_price(symbol=JPM)                                                                          ┃
┃  - get_company_news(symbol=JPM)                                                                                 ┃
┃                                                                                                                 ┃
┃ The current stock price for JPM is $269.35. Recent news stories related to JPM include:                         ┃
┃                                                                                                                 ┃
┃ 1. "Companies like Costco and Apple are defending their DEI programs despite nationwide pushback\u2014These are ┃
┃ the companies standing by their policies" - discusses companies maintaining their diversity, equity, and        ┃
┃ inclusion (DEI) policies despite opposition.                                                                    ┃
┃ 2. "Global Euphoria for Indian Bonds Wanes as Rupee Slumps" - discusses the state of the Indian bond market.    ┃
┃ 3. "Is JPMorgan Chase & Co. (JPM) Unstoppable Dividend Stock to Invest In?" - analyzes JPMorgan Chase as an     ┃
┃ investment opportunity, suggesting it is an unstoppable dividend stock.                                         ┃
┃                                                                                                                 ┃
┃ These news stories may provide some insight into the current state and trends of JPM. For a more comprehensive  ┃
┃ trend analysis, you might want to consult financial news outlets or analyst reports.                            ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛